In [1]:
# Import libraries
import requests
from bs4 import BeautifulSoup
import re
import csv

In [11]:
# Collect first page of list
# list_of_pages = requests.get("https://www.immobilienscout24.de/Suche/S-T/Wohnung-Miete/Fahrzeitsuche/Frankfurt_20am_20Main/-/-95993/2244895/-/1276007004/60?enteredFrom=result_list")

# Create a BeautifulSoup object
# soup_for_list = BeautifulSoup(list_of_pages.text, 'html.parser')


result = []
header = ["type_of_house", "living_space", "number_of_rooms", "rent", "construction_year", "street", "zipcode", "region", "tag"]
result.append(header)

def page_spider():
    for i in range(0, 143):
        link = "https://www.immobilienscout24.de/Suche/S-"
        link += str(i)
        link += "/Wohnung-Miete/Fahrzeitsuche/Frankfurt_20am_20Main/-/-95993/2244895/-/1276007004/60?enteredFrom=result_list"
        page = requests.get(link)
        soup_for_list = BeautifulSoup(page.text, 'html.parser')
        list_spider(soup_for_list)

def list_spider(soup_for_list):
    lst = []
    for paragraph in soup_for_list.find_all('a'):
        if '/expose/' in str(paragraph.get("href")):
            lst.append(paragraph.get('href'))

    lst = list(set(lst))

    for item in lst:
        page = requests.get("https://www.immobilienscout24.de"+item)
        soup = BeautifulSoup(page.text, 'html.parser')

        try:
            type_of_house = soup.find(class_= 'is24qa-typ grid-item three-fifths').text.strip()
        except:
            type_of_house = ''
        try:
            living_space = soup.find(class_='is24qa-wohnflaeche-ca').text.strip()
        except:
            living_space = ''
        try:
            number_of_rooms = int(soup.find(class_='is24qa-zimmer grid-item three-fifths').text.strip())
        except:
            number_of_rooms = ''
        try:
            rent = soup.find(class_='is24qa-gesamtmiete').text.strip()
        except:
            rent = ''
        try:
            construction_year = soup.find(class_='is24qa-baujahr').text.strip()
        except:
            construction_year = ''
        try:
            street = soup.find(class_='block font-nowrap print-hide').text.strip().rstrip(',')
        except:
            street = ''
        try:
            zip_region = soup.find(class_='zip-region-and-country').text.strip()
            zipcode = re.sub('[^\d]+', '', zip_region).strip()
            region = re.sub('[\d]+', '', zip_region).strip()
        except:
            zipcode = ''
            region = ''

        living_space = int(re.sub('[^\d,]+','',living_space).split(",")[0])
        rent = int(re.sub('[^\d,]+', '', rent).split(",")[0])

        new_list = [type_of_house, living_space, number_of_rooms, rent, construction_year, street, zipcode, region, item]
        result.append(new_list)

    with open("data.csv", "w", newline="") as f:
        writer = csv.writer(f)
        writer = writer.writerows(result)

page_spider()

In [4]:
import requests 
import urllib.parse
import pandas as pd 
from time import sleep

df = pd.read_csv("data.csv", nrows = 2450)
URL1 = 'https://api.opencagedata.com/geocode/v1/json?q='
URL3 = '&key=71b9900ab09246f99610ef8cfb212c2e&language=en&pretty=1'
lst_lat = []
lst_lng = []

for index, row in df.iterrows():
    query = str(row['street']) +  ", " + str(row['zipcode']) + ", " + str(row['region'])
    URL2 = urllib.parse.quote(query)
    r = requests.get(URL1+URL2+URL3)
    data = r.json()
    try: 
        latitude = data['results'][0]['geometry']['lat']
        longitude = data['results'][0]['geometry']['lng']
    except: 
        latitude = ''
        longitude = ''
    lst_lat.append(latitude)
    lst_lng.append(longitude)
    sleep(1)
df["latitude"] = lst_lat
df["longitude"] = lst_lat
df


0 50.1221932 8.6435081
1 50.1176897 8.6390767
2 50.1197128 8.6412505
3 50.11552 8.68417
4 50.1167958 8.6369873
5 50.1223606 8.637532
6 50.116794 8.6373408
7 50.1226603 8.6455341
8 50.11552 8.68417
9 50.1174939 8.6354119
10 50.1176404 8.6335523
11 50.11552 8.68417
12 50.1179512 8.6358194
13 50.1176404 8.6335523
14 50.1223606 8.637532
15 50.1223606 8.637532
16 50.122509 8.6350992
17 50.1195267 8.6460675
18 50.1221932 8.6435081
19 50.1226603 8.6455341
20 50.1221932 8.6435081
21 50.1176897 8.6390767
22 50.1197128 8.6412505
23 50.11552 8.68417
24 50.1167958 8.6369873
25 50.1223606 8.637532
26 50.116794 8.6373408
27 50.1226603 8.6455341
28 50.11552 8.68417
29 50.1174939 8.6354119
30 50.1176404 8.6335523
31 50.11552 8.68417
32 50.1179512 8.6358194
33 50.1176404 8.6335523
34 50.1223606 8.637532
35 50.1223606 8.637532
36 50.122509 8.6350992
37 50.1195267 8.6460675
38 50.1221932 8.6435081
39 50.1226603 8.6455341
40 50.0870527 8.6396668
41 50.1580146 8.7620388
42 50.006379 8.2853675
43 50.1169599

363 50.1169599 8.7069227
364 50.1139623 8.6976778
365 50.1785375 8.4666394
366 50.0044913 8.5817868
367 50.0022184 8.2689231
368 50.1045694 8.705045
369 50.1008277 8.6609364
370 50.2313417 8.7717673
371 50.1249136 8.67795
372 50.1804837 8.7410776
373  
374  
375 50.1127309 8.6932084
376  
377 50.0817745 8.6313639
378 50.0817745 8.6313639
379 49.8588353 8.6465863
380 50.0870527 8.6396668
381 50.1429142 8.5029259
382 50.1580146 8.7620388
383 50.1169599 8.7069227
384 50.1139623 8.6976778
385 50.1785375 8.4666394
386 50.0044913 8.5817868
387 50.0022184 8.2689231
388 50.1045694 8.705045
389 50.1008277 8.6609364
390 50.2313417 8.7717673
391 50.1249136 8.67795
392 50.1804837 8.7410776
393  
394  
395 50.1127309 8.6932084
396  
397 50.0817745 8.6313639
398 50.0817745 8.6313639
399 49.8588353 8.6465863
400 50.0870527 8.6396668
401 50.1429142 8.5029259
402 50.1580146 8.7620388
403 50.1169599 8.7069227
404 50.1139623 8.6976778
405 50.1785375 8.4666394
406 50.0044913 8.5817868
407 50.0022184 8.268

738 50.1221932 8.6435081
739 50.1176897 8.6390767
740 50.1197128 8.6412505
741 50.11552 8.68417
742 50.1167958 8.6369873
743 50.1223606 8.637532
744 50.116794 8.6373408
745 50.1226603 8.6455341
746 50.11552 8.68417
747 50.1174939 8.6354119
748 50.1176404 8.6335523
749 50.11552 8.68417
750 50.1179512 8.6358194
751 50.1176404 8.6335523
752 50.1223606 8.637532
753 50.1223606 8.637532
754 50.122509 8.6350992
755 50.1195267 8.6460675
756 50.1221932 8.6435081
757 50.1226603 8.6455341
758 50.1221932 8.6435081
759 50.1176897 8.6390767
760 50.1197128 8.6412505
761 50.11552 8.68417
762 50.1167958 8.6369873
763 50.1223606 8.637532
764 50.116794 8.6373408
765 50.1226603 8.6455341
766 50.11552 8.68417
767 50.1174939 8.6354119
768 50.1176404 8.6335523
769 50.11552 8.68417
770 50.1179512 8.6358194
771 50.1176404 8.6335523
772 50.1223606 8.637532
773 50.1223606 8.637532
774 50.122509 8.6350992
775 50.1195267 8.6460675
776 50.1221932 8.6435081
777 50.1226603 8.6455341
778 50.1221932 8.6435081
779 50.11

1075 50.1195267 8.6460675
1076 50.1221932 8.6435081
1077 50.1226603 8.6455341
1078 50.1221932 8.6435081
1079 50.1176897 8.6390767
1080 50.1197128 8.6412505
1081 50.11552 8.68417
1082 50.1167958 8.6369873
1083 50.1223606 8.637532
1084 50.116794 8.6373408
1085 50.1226603 8.6455341
1086 50.11552 8.68417
1087 50.1174939 8.6354119
1088 50.1176404 8.6335523
1089 50.11552 8.68417
1090 50.1179512 8.6358194
1091 50.1176404 8.6335523
1092 50.1223606 8.637532
1093 50.1223606 8.637532
1094 50.122509 8.6350992
1095 50.1195267 8.6460675
1096 50.1221932 8.6435081
1097 50.1226603 8.6455341
1098 50.1221932 8.6435081
1099 50.1176897 8.6390767
1100 50.1197128 8.6412505
1101 50.11552 8.68417
1102 50.1167958 8.6369873
1103 50.1223606 8.637532
1104 50.116794 8.6373408
1105 50.1226603 8.6455341
1106 50.11552 8.68417
1107 50.1174939 8.6354119
1108 50.1176404 8.6335523
1109 50.11552 8.68417
1110 50.1179512 8.6358194
1111 50.1176404 8.6335523
1112 50.1223606 8.637532
1113 50.1223606 8.637532
1114 50.122509 8.63

1401 50.11552 8.68417
1402 50.1167958 8.6369873
1403 50.1223606 8.637532
1404 50.116794 8.6373408
1405 50.1226603 8.6455341
1406 50.11552 8.68417
1407 50.1174939 8.6354119
1408 50.1176404 8.6335523
1409 50.11552 8.68417
1410 50.1179512 8.6358194
1411 50.1176404 8.6335523
1412 50.1223606 8.637532
1413 50.1223606 8.637532
1414 50.122509 8.6350992
1415 50.1195267 8.6460675
1416 50.1221932 8.6435081
1417 50.1226603 8.6455341
1418 50.1221932 8.6435081
1419 50.1176897 8.6390767
1420 50.1197128 8.6412505
1421 50.11552 8.68417
1422 50.1167958 8.6369873
1423 50.1223606 8.637532
1424 50.116794 8.6373408
1425 50.1226603 8.6455341
1426 50.11552 8.68417
1427 50.1174939 8.6354119
1428 50.1176404 8.6335523
1429 50.11552 8.68417
1430 50.1179512 8.6358194
1431 50.1176404 8.6335523
1432 50.1223606 8.637532
1433 50.1223606 8.637532
1434 50.122509 8.6350992
1435 50.1195267 8.6460675
1436 50.1221932 8.6435081
1437 50.1226603 8.6455341
1438 50.1221932 8.6435081
1439 50.1176897 8.6390767
1440 50.1197128 8.64

1727 50.1174939 8.6354119
1728 50.1176404 8.6335523
1729 50.11552 8.68417
1730 50.1179512 8.6358194
1731 50.1176404 8.6335523
1732 50.1223606 8.637532
1733 50.1223606 8.637532
1734 50.122509 8.6350992
1735 50.1195267 8.6460675
1736 50.1221932 8.6435081
1737 50.1226603 8.6455341
1738 50.1221932 8.6435081
1739 50.1176897 8.6390767
1740 50.1197128 8.6412505
1741 50.11552 8.68417
1742 50.1167958 8.6369873
1743 50.1223606 8.637532
1744 50.116794 8.6373408
1745 50.1226603 8.6455341
1746 50.11552 8.68417
1747 50.1174939 8.6354119
1748 50.1176404 8.6335523
1749 50.11552 8.68417
1750 50.1179512 8.6358194
1751 50.1176404 8.6335523
1752 50.1223606 8.637532
1753 50.1223606 8.637532
1754 50.122509 8.6350992
1755 50.1195267 8.6460675
1756 50.1221932 8.6435081
1757 50.1226603 8.6455341
1758 50.1221932 8.6435081
1759 50.1176897 8.6390767
1760 50.1197128 8.6412505
1761 50.11552 8.68417
1762 50.1167958 8.6369873
1763 50.1223606 8.637532
1764 50.116794 8.6373408
1765 50.1226603 8.6455341
1766 50.11552 8.

2053 50.1223606 8.637532
2054 50.122509 8.6350992
2055 50.1195267 8.6460675
2056 50.1221932 8.6435081
2057 50.1226603 8.6455341
2058 50.1221932 8.6435081
2059 50.1176897 8.6390767
2060 50.1197128 8.6412505
2061 50.11552 8.68417
2062 50.1167958 8.6369873
2063 50.1223606 8.637532
2064 50.116794 8.6373408
2065 50.1226603 8.6455341
2066 50.11552 8.68417
2067 50.1174939 8.6354119
2068 50.1176404 8.6335523
2069 50.11552 8.68417
2070 50.1179512 8.6358194
2071 50.1176404 8.6335523
2072 50.1223606 8.637532
2073 50.1223606 8.637532
2074 50.122509 8.6350992
2075 50.1195267 8.6460675
2076 50.1221932 8.6435081
2077 50.1226603 8.6455341
2078 50.1221932 8.6435081
2079 50.1176897 8.6390767
2080 50.1197128 8.6412505
2081 50.11552 8.68417
2082 50.1167958 8.6369873
2083 50.1223606 8.637532
2084 50.116794 8.6373408
2085 50.1226603 8.6455341
2086 50.11552 8.68417
2087 50.1174939 8.6354119
2088 50.1176404 8.6335523
2089 50.11552 8.68417
2090 50.1179512 8.6358194
2091 50.1176404 8.6335523
2092 50.1223606 8.6

2379 50.1176897 8.6390767
2380 50.1197128 8.6412505
2381 50.11552 8.68417
2382 50.1167958 8.6369873
2383 50.1223606 8.637532
2384 50.116794 8.6373408
2385 50.1226603 8.6455341
2386 50.11552 8.68417
2387 50.1174939 8.6354119
2388 50.1176404 8.6335523
2389 50.11552 8.68417
2390 50.1179512 8.6358194
2391 50.1176404 8.6335523
2392 50.1223606 8.637532
2393 50.1223606 8.637532
2394 50.122509 8.6350992
2395 50.1195267 8.6460675
2396 50.1221932 8.6435081
2397 50.1226603 8.6455341
2398 50.1221932 8.6435081
2399 50.1176897 8.6390767
2400 50.1197128 8.6412505
2401 50.11552 8.68417
2402 50.1167958 8.6369873
2403 50.1223606 8.637532
2404 50.116794 8.6373408
2405 50.1226603 8.6455341
2406 50.11552 8.68417
2407 50.1174939 8.6354119
2408 50.1176404 8.6335523
2409 50.11552 8.68417
2410 50.1179512 8.6358194
2411 50.1176404 8.6335523
2412 50.1223606 8.637532
2413 50.1223606 8.637532
2414 50.122509 8.6350992
2415 50.1195267 8.6460675
2416 50.1221932 8.6435081
2417 50.1226603 8.6455341
2418 50.1221932 8.64

,type_of_house,living_space,number_of_rooms,rent,construction_year,street,zipcode,region,tag,latitude,longitude
0,Etagenwohnung,62,2.0,1350,2011,Kurfürstenstr. 14a,60486,"Frankfurt am Main, Bockenheim",/expose/61973555,50.1222,50.1222
1,Etagenwohnung,45,1.0,205,unbekannt,Pfingstbrunnenstraße 40,60486,"Frankfurt, Bockenheim",/expose/83854266,50.1177,50.1177
2,Dachgeschoss,80,3.0,1550,1880,Kurfürstenstr. 49,60486,"Frankfurt am Main, Bockenheim",/expose/114237680,50.1197,50.1197
3,NaN,60,2.0,1040,unbekannt,Fritzlarer Strasse 9,60487,"Frankfurt a.M. - Bockenheim, Frankfurt am Main",/expose/114435895,50.1155,50.1155
4,Etagenwohnung,81,3.0,1445,2005,Voltastraße 43,60486,"Frankfurt am Main, Bockenheim",/expose/114546550,50.1168,50.1168
5,Dachgeschoss,20,1.0,440,1965,NaN,60486,"Frankfurt, Bockenheim",/expose/82913104,50.1224,50.1224
6,Etagenwohnung,83,2.0,1445,2005,Voltastraße 41,60486,"Frankfurt am Main, Bockenheim",/expose/113486807,50.1168,50.1168
7,Etagenwohnung,100,2.0,2080,2010,Leipziger Straße 00,60486,"Frankfurt-Bockenheim, Frankfurt am Main",/expose/113395106,50.1227,50.1227
8,NaN,60,2.0,1040,unbekannt,Fritzlarer Strasse 9,60487,"Frankfurt a.M. - Bockenheim, Frankfurt am Main",/expose/114435845,50.1155,50.1155
9,Maisonette,115,3.0,1950,2009,Ohmstraße 55,60486,"Frankfurt am Main, Bockenheim",/expose/114348636,50.1175,50.1175


In [5]:
df.to_csv("new_data1.csv", sep='\t')

In [6]:
import pandas as pd 
df = pd.read_csv("new_data1.csv", sep='\t', nrows = 2450)
df

,Unnamed: 0,type_of_house,living_space,number_of_rooms,rent,construction_year,street,zipcode,region,tag,latitude,longitude
0,0,Etagenwohnung,62,2.0,1350,2011,Kurfürstenstr. 14a,60486,"Frankfurt am Main, Bockenheim",/expose/61973555,50.122193,50.122193
1,1,Etagenwohnung,45,1.0,205,unbekannt,Pfingstbrunnenstraße 40,60486,"Frankfurt, Bockenheim",/expose/83854266,50.117690,50.117690
2,2,Dachgeschoss,80,3.0,1550,1880,Kurfürstenstr. 49,60486,"Frankfurt am Main, Bockenheim",/expose/114237680,50.119713,50.119713
3,3,NaN,60,2.0,1040,unbekannt,Fritzlarer Strasse 9,60487,"Frankfurt a.M. - Bockenheim, Frankfurt am Main",/expose/114435895,50.115520,50.115520
4,4,Etagenwohnung,81,3.0,1445,2005,Voltastraße 43,60486,"Frankfurt am Main, Bockenheim",/expose/114546550,50.116796,50.116796
5,5,Dachgeschoss,20,1.0,440,1965,NaN,60486,"Frankfurt, Bockenheim",/expose/82913104,50.122361,50.122361
6,6,Etagenwohnung,83,2.0,1445,2005,Voltastraße 41,60486,"Frankfurt am Main, Bockenheim",/expose/113486807,50.116794,50.116794
7,7,Etagenwohnung,100,2.0,2080,2010,Leipziger Straße 00,60486,"Frankfurt-Bockenheim, Frankfurt am Main",/expose/113395106,50.122660,50.122660
8,8,NaN,60,2.0,1040,unbekannt,Fritzlarer Strasse 9,60487,"Frankfurt a.M. - Bockenheim, Frankfurt am Main",/expose/114435845,50.115520,50.115520
9,9,Maisonette,115,3.0,1950,2009,Ohmstraße 55,60486,"Frankfurt am Main, Bockenheim",/expose/114348636,50.117494,50.117494


In [6]:
df.to_csv("new_data2.csv", sep='\t', encoding='utf-8')

In [ ]:
# wc -l data.csv